El primer paso del tutorial es instalar todas las librerias y dependencias necesarias.

In [33]:
%pip install --quiet --upgrade langchain langchain-community langchain-chroma
%pip install -qU langchain-openai
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


A continuación se establece la llave de acceso a la API de LanchChain

In [ ]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = ""

Se conecta la clave de acceso a chat-gpt y se elige el modelo de lenguaje, en este caso got-4o-mini

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = ""

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

En este fragmento se carga y se extrae un contenido específico de un post ubicado en una página web. El método bs4.SoupStrainer crea un filto para obtener únicamente las partes del HTML relevantes y el método WebBaseLoader obtiene el contenido de la URL especificada y las filtra con el método mencionado anteriormente.

In [17]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

len(docs[0].page_content)

43131

En esta linea se imprimen las primeras 500 líneas del documento obtenido de la URL

In [18]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


Como el documento obtenido tiene mas de 42.000 caracteres necesita dividirse en fragmentos pues es muy extenso para su analisis completo, por lo cuál se dividirá en fragmentos de 1000 caracteres con una superposición de 200 caracteres para que se mantenga el contexto y conexiones lógicas entre los fragmentos.

In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

66

Aquí se calcula la longitud en caracteres del contenido del primer fragmento, que es de 969 porque se respeta los finales e inicios de palabras y párrafos para mantener la estructura y lógica del texto

In [20]:
len(all_splits[0].page_content)

969

En la siguiente línea se obtienen los metadatos del fragmento en la posición 10, contiene el índice de inicio del fragmento en el documento original, que indica en qué posición del texto completo comienza este fragmento

In [21]:
all_splits[10].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'start_index': 7056}

En este bloque se crea un almacén vectorial a partir de los fragmentos del documento, es decir convierte el texto en vectores numéricos

In [22]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

A continuación se utiliza el almacén vectorial creado y se realiza una búsqueda de similitud en función de la consulta que se le pasó: "What are the approaches to Task Decomposition?". Además se le especifica que debe obtener los 6 documentos que más se asemejan a la búsqueda

In [23]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")

len(retrieved_docs)

6

Aquí se imprime el contenido de la página del primer documento de la consulta realizada

In [24]:
print(retrieved_docs[0].page_content)

Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.


En esta parte se establece la conexión con el modelo gtp-40-mini a parte de la api key suminstrada

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = ""

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

A continuación se obtiene un prompt predefinido por LanchChain

In [26]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:", additional_kwargs={}, response_metadata={})]

Aquí se muestra el prompt que se está estableciendo 

In [27]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


Se define una función que toma la lista de comuentos y los convierte en una cadena de texto, uniendo el contenido de cada documento (doc.page_content) con saltos de línea dobles. Luego se define una cadena de procesamiento que primero toma el contexto recuperado por el retriever, lo pasa por format_docs para convertirlo en un formato de texto y lo asigna a la clave "context". La clave "question" recibe un RunnablePassthrough(), lo que significa que la pregunta proporcionada se pasa tal cual a la siguiente etapa sin alteración. Luego el contexto formateado y la pregunta pasan al prompt previamente cargado desde LangChain, este resultado se pasa al modelo de lenguaje que generará la respuesta basada en la consulta y el contexto, finalmente StrOutputParser se encarga de extraer el texto de la respuesta generada por el modelo.

In [28]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

Task decomposition is the process of breaking down complex tasks into smaller, more manageable steps. It can be facilitated by techniques like Chain of Thought (CoT) and Tree of Thoughts, which guide models to think step by step and explore multiple reasoning possibilities. This method enhances performance by allowing clearer planning and execution of tasks.

A continuación se le pasa el prompt el cuál le especifica que el modelo debe usar el contexto recuperado para responder la pregunta de manera concisa (máximo de tres oraciones). Si no sabe la respuesta, debe indicar que no sabe. Luego estructura dicho prompt en el mensaje que establece cómo debe comportase el modelo y la otra parte que es el mensaje del usuario que contiene la pregunta que se le hace al modelo. Luego crea una cadena que combina los documentos recuperados con el modelo de lenguaje (llm) y el prompt para generar respuestas basadas en el contexto de los documentos. Esta cadena toma el contexto (documentos) y la pregunta como entrada, pasando el contexto al modelo de lenguaje junto con la pregunta. Posteriormente create_retrieval_chain crea una cadena que combina el recuperador de documentos (retriever) con la cadena de pregunta-respuesta (question_answer_chain). Esto permite que el retriever recupere los documentos más relevantes basados en la pregunta y luego los pase a la cadena de generación de respuestas para producir una respuesta concisa. Luego rag_chain.invoke({"input": "What is Task Decomposition?"}): Llama a la cadena rag_chain con la pregunta "What is Task Decomposition?" y obtiene la respuesta generada, que se almacena en response. El parámetro {"input": "What is Task Decomposition?"} es el diccionario que contiene la pregunta. Finalmente imprime la respuesta generada por el modelo.

In [30]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is Task Decomposition?"})
print(response["answer"])

Task decomposition is the process of breaking down a complicated task into smaller, more manageable steps. Techniques like Chain of Thought (CoT) and Tree of Thoughts (ToT) are used to help models reason through tasks by generating step-by-step thoughts and exploring multiple reasoning possibilities. This approach enhances model performance and provides insights into the model's thinking process.


En este bloque de código se obtiene la lista de comentos o fragmentos de texto que se usaron como contexto para generar la respuesta.

In [31]:
for document in response["context"]:
    print(document)
    print()

page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.' metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 1585}

page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS 

Aquí se define una plantilla para el prompt. PromptTemplate.from_template(template) crea un objeto PromptTemplate al cuál se le inserta dicha plantilla. Luego se define rag_chain que es toda la cadena de procesamiento descrita más arriba. Finalmente se invoca la cadena con la pregunta que se quiere hacer.

In [32]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

"Task Decomposition is the process of breaking down a complex task into smaller, manageable steps to make it easier to execute. It can be achieved using techniques like Chain of Thought (CoT) or Tree of Thoughts, which guide the model in thinking step-by-step or exploring multiple reasoning possibilities. This approach enhances the model's performance and clarity in problem-solving. Thanks for asking!"